#Analyzing physical activity monitor data

TODO intro

## Data

TODO

To convert from sas transport file format with extension`.xpt` to CSV we can use `xport` module from python PyPI packages:
```bash
pip install xport
```
and use the `xport` module as a command-line tool to convert an XPT file to CSV file:

```bash
python -m xport paxraw_d.xpt > paxraw_d.csv
```

## Reading the data

In [ ]:
val spark = sparkSession

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6520cbdf


In [ ]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._


In [ ]:
val PaxSchema = StructType(
    StructField("SEQN", FloatType, false) ::
    StructField("PAXSTAT", FloatType, false) ::
    StructField("PAXCAL", FloatType, false) ::
    StructField("PAXDAYSAS", FloatType, false) ::
    StructField("PAXN", FloatType, false) ::
    StructField("PAXHOUR", FloatType, false) ::
    StructField("PAXMINUT", FloatType, false) ::
    StructField("PAXINTEN", FloatType, false) ::
    StructField("PAXSTEP", StringType, true) :: Nil
)

PaxSchema: org.apache.spark.sql.types.StructType = StructType(StructField(SEQN,FloatType,false), StructField(PAXSTAT,FloatType,false), StructField(PAXCAL,FloatType,false), StructField(PAXDAYSAS,FloatType,false), StructField(PAXN,FloatType,false), StructField(PAXHOUR,FloatType,false), StructField(PAXMINUT,FloatType,false), StructField(PAXINTEN,FloatType,false), StructField(PAXSTEP,StringType,true))


TODO note on `nan` values in `PAXSTEP` column

In [ ]:
val PaxDF = spark.read
  .format("csv")
  .schema(PaxSchema)
  .option("header", true)
  .load("./notebooks/spark-notebooks-gallery/gallery/physical-activity-monitor/data/paxraw_d.csv")
  .withColumn("PAXSTEP", when($"PAXSTEP" === "nan", null).otherwise($"PAXSTEP".cast(DoubleType)))
  .select($"SEQN".cast(IntegerType),
          $"PAXSTAT".cast(IntegerType),
          $"PAXCAL".cast(IntegerType),
          $"PAXDAYSAS".cast(IntegerType),
          $"PAXN".cast(IntegerType),
          $"PAXHOUR".cast(IntegerType),
          $"PAXMINUT".cast(IntegerType),
          $"PAXINTEN".cast(IntegerType),
          $"PAXSTEP".cast(IntegerType))
  .filter(!isnull($"PAXSTEP"))

PaxDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [SEQN: int, PAXSTAT: int ... 7 more fields]


In [ ]:
PaxDF.write.format("parquet").save("./notebooks/spark-notebooks-gallery/gallery/physical-activity-monitor/data/paxraw_d.parquet")

In [ ]:
PaxDF.count

res18: Long = 74874033


74874033

In [ ]:
val PaxDF = spark.read
  .format("parquet")
  //.schema(PaxSchema)
  .load("./notebooks/spark-notebooks-gallery/gallery/physical-activity-monitor/data/paxraw_d.parquet")

PaxDF: org.apache.spark.sql.DataFrame = [SEQN: int, PAXSTAT: int ... 7 more fields]


In [ ]:
PaxDF.count

res25: Long = 74874033


74874033